In [23]:
import polars as pl

df = pl.read_parquet('./data/outputs/play_by_play_agg/play_by_play_agg_20240809.parquet')

In [24]:

df

game_id,team,home_away,opponent,player,player_id,passing_attempts,passing_yards,passing_touchdowns,rushing_attempts,rushing_yards,rushing_touchdowns,receptions,receiving_yards,receiving_touchdowns,fumbles,interceptions,season,week,position,curr_team
str,str,str,str,str,str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],i64,i64,str,str
"""2022_03_GB_TB""","""GB""","""away""","""GB""","""A.Dillon""","""00-0036265""","[""passing_attempts"", ""sum""]","[""passing_yards"", ""sum""]","[""passing_touchdowns"", ""sum""]","[""rushing_attempts"", ""sum""]","[""rushing_yards"", ""sum""]","[""rushing_touchdowns"", ""sum""]","[""receptions"", ""sum""]","[""receiving_yards"", ""sum""]","[""receiving_touchdowns"", ""sum""]","[""fumble"", ""sum""]","[""interception"", ""sum""]",2022,3,"""RB""","""GB"""
"""2020_04_NO_DET""","""NO""","""away""","""NO""","""L.Murray""","""00-0030513""","[""passing_attempts"", ""sum""]","[""passing_yards"", ""sum""]","[""passing_touchdowns"", ""sum""]","[""rushing_attempts"", ""sum""]","[""rushing_yards"", ""sum""]","[""rushing_touchdowns"", ""sum""]","[""receptions"", ""sum""]","[""receiving_yards"", ""sum""]","[""receiving_touchdowns"", ""sum""]","[""fumble"", ""sum""]","[""interception"", ""sum""]",2020,4,"""RB""","""BUF"""
"""2023_14_HOU_NYJ""","""HOU""","""away""","""HOU""","""J.Metchie""","""00-0037614""","[""passing_attempts"", ""sum""]","[""passing_yards"", ""sum""]","[""passing_touchdowns"", ""sum""]","[""rushing_attempts"", ""sum""]","[""rushing_yards"", ""sum""]","[""rushing_touchdowns"", ""sum""]","[""receptions"", ""sum""]","[""receiving_yards"", ""sum""]","[""receiving_touchdowns"", ""sum""]","[""fumble"", ""sum""]","[""interception"", ""sum""]",2023,14,"""WR""","""HOU"""
"""2021_02_DAL_LAC""","""DAL""","""away""","""DAL""","""C.Wilson""","""00-0034418""","[""passing_attempts"", ""sum""]","[""passing_yards"", ""sum""]","[""passing_touchdowns"", ""sum""]","[""rushing_attempts"", ""sum""]","[""rushing_yards"", ""sum""]","[""rushing_touchdowns"", ""sum""]","[""receptions"", ""sum""]","[""receiving_yards"", ""sum""]","[""receiving_touchdowns"", ""sum""]","[""fumble"", ""sum""]","[""interception"", ""sum""]",2021,2,"""WR""","""NO"""
"""2023_08_HOU_CAR""","""HOU""","""away""","""HOU""","""D.Pierce""","""00-0037258""","[""passing_attempts"", ""sum""]","[""passing_yards"", ""sum""]","[""passing_touchdowns"", ""sum""]","[""rushing_attempts"", ""sum""]","[""rushing_yards"", ""sum""]","[""rushing_touchdowns"", ""sum""]","[""receptions"", ""sum""]","[""receiving_yards"", ""sum""]","[""receiving_touchdowns"", ""sum""]","[""fumble"", ""sum""]","[""interception"", ""sum""]",2023,8,"""RB""","""HOU"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2023_17_NE_BUF""","""BUF""","""home""","""BUF""","""D.Knox""","""00-0035689""","[""passing_attempts"", ""sum""]","[""passing_yards"", ""sum""]","[""passing_touchdowns"", ""sum""]","[""rushing_attempts"", ""sum""]","[""rushing_yards"", ""sum""]","[""rushing_touchdowns"", ""sum""]","[""receptions"", ""sum""]","[""receiving_yards"", ""sum""]","[""receiving_touchdowns"", ""sum""]","[""fumble"", ""sum""]","[""interception"", ""sum""]",2023,17,"""TE""","""BUF"""
"""2020_16_DEN_LAC""","""DEN""","""away""","""DEN""","""K.Hamler""","""00-0036412""","[""passing_attempts"", ""sum""]","[""passing_yards"", ""sum""]","[""passing_touchdowns"", ""sum""]","[""rushing_attempts"", ""sum""]","[""rushing_yards"", ""sum""]","[""rushing_touchdowns"", ""sum""]","[""receptions"", ""sum""]","[""receiving_yards"", ""sum""]","[""receiving_touchdowns"", ""sum""]","[""fumble"", ""sum""]","[""interception"", ""sum""]",2020,16,"""WR""","""BUF"""
"""2023_07_DET_BAL""","""BAL""","""home""","""BAL""","""J.Hill""","""00-0034975""","[""passing_attempts"", ""sum""]","[""passing_yards"", ""sum""]","[""passing_touchdowns"", ""sum""]","[""rushing_attempts"", ""sum""]","[""rushing_yards"", ""sum""]","[""rushing_touchdowns"", ""sum""]","[""receptions"", "